# 🚀 ShapeKeyFaceTracker - Windows EXE Builder (Wine 10 Fix)

แก้ไขปัญหา `api-ms-win-crt` แครชโดยการใช้ **PyInstaller 5.13.2** (เสถียรกว่าบน Wine) และบังคับใช้ Windows 10 ในระบบ Wine อย่างเข้มงวด

### 1️⃣ Setup Wine & Windows 10 Environment

In [ ]:
# ติดตั้ง Wine
!sudo dpkg --add-architecture i386
!sudo apt-get update
!sudo apt-get install -y wine64 wine32 xvfb zip unzip

# ล้างค่าเก่าและตั้งค่า Wine Prefix ใหม่
!rm -rf /root/.wine64
%env WINEARCH=win64
%env WINEPREFIX=/root/.wine64
%env WINEDEBUG=-all

# บังคับจดจำ Windows 10
!xvfb-run winecfg -v win10
!sleep 2
!xvfb-run wineboot -i

### 2️⃣ Download Portable Windows Python

In [ ]:
# ดาวน์โหลด Python Standalone
!wget -q --show-progress https://github.com/indygreg/python-build-standalone/releases/download/20230507/cpython-3.10.11+20230507-x86_64-pc-windows-msvc-shared-install_only.tar.gz

# แตกไฟล์
!mkdir -p /opt/python-windows
!tar -xf cpython-3.10.11+20230507-x86_64-pc-windows-msvc-shared-install_only.tar.gz -C /opt/python-windows

### 3️⃣ Install Dependencies (Safe Version)
*(ใช้ PyInstaller 5.x เพื่อหลีกเลี่ยงระบบ Isolated Hook ที่ชอบแครชบน Wine)*

In [ ]:
WINE_PYTHON = "/opt/python-windows/python/python.exe"

print("📦 Installing Safe Dependencies...")
!xvfb-run wine {WINE_PYTHON} -m ensurepip
!xvfb-run wine {WINE_PYTHON} -m pip install --upgrade pip
!xvfb-run wine {WINE_PYTHON} -m pip install dearpygui opencv-python mediapipe numpy
# ใช้ 5.13.2 เพราะเสถียรที่สุดบน Wine ในปัจจุบัน
!xvfb-run wine {WINE_PYTHON} -m pip install pyinstaller==5.13.2

### 4️⃣ Build the EXE
*(อัปโหลด main_dpg.py และ assets ไว้ที่หน้าแรกของ Colab)*

In [ ]:
import os
WINE_PYTHON = "/opt/python-windows/python/python.exe"
MP_PATH = "/opt/python-windows/python/lib/site-packages/mediapipe"

if not os.path.exists("main_dpg.py"):
    print("❌ กรุณาอัปโหลด main_dpg.py ก่อน!")
else:
    print("🛠 เริ่ม Build EXE (Stable Build Mode)...")
    
    # เพิ่มค่า ENV เล็กน้อยเพื่อช่วย Wine
    # ใช้ manual mode ของการดึง mediapipe มาเลย
    !xvfb-run wine {WINE_PYTHON} -m PyInstaller --onefile --noconsole \
        --name "ShapeKeyFaceTracker" \
        --add-data "face_landmarker.task;." \
        --add-data "face_mesh.png;." \
        --add-data "{MP_PATH};mediapipe/" \
        --hidden-import "mediapipe.python._framework_bindings" \
        main_dpg.py

    print("\n✨ Build เสร็จสิ้น! ถ้าไม่มีข้อความ Error ขั้นรุนแรง ให้เช็คในโฟลเดอร์ dist")

### 5️⃣ Download result

In [ ]:
from google.colab import files
import os

exe_path = "dist/ShapeKeyFaceTracker.exe"
if os.path.exists(exe_path):
    files.download(exe_path)
else:
    print("❌ ไม่พบไฟล์ EXE. หากเจอ error unimplemented function ให้ลองกดรัน Cell 1 อีกรอบแล้วค่อยรัน Cell 4")
